In [ ]:

from PIL import Image
import numpy as np

def img_to_arr(img_path):
    with Image.open(img_path) as img:
        img_arr = np.array(img, dtype=np.uint8)
        img_arr = np.where(img_arr==255, 1, img_arr)
    return img_arr

def arr_to_img(img_new, save_path):
    img_new = np.where(img_new==1, 255, img_new)
    result_img = Image.fromarray(img_new)
    result_img.save(save_path)

def erode(img_arr, se):
    irow, icol = img_arr.shape[0], img_arr.shape[1]
    img_new = np.zeros((irow, icol), dtype=np.uint8)
    mrow, mcol = se.shape[0], se.shape[1]
    for i in range(mrow//2, irow-mrow//2):
        for j in range(mcol//2, icol-mcol//2):
            img_new[i, j] = 1
            for k in range(mrow):
                for l in range(mcol):
                    if se[k][l]==1 and img_arr[i-mrow//2+k, j-mcol//2+l]==0:
                        img_new[i, j] = 0
                        break
                else:
                    continue
                break
    return img_new

def dilate(img_arr, se):
    irow, icol = img_arr.shape[0], img_arr.shape[1]
    img_new = np.zeros((irow, icol), dtype=np.uint8)
    mrow, mcol = se.shape[0], se.shape[1]
    for i in range(mrow//2, irow-mrow//2):
        for j in range(mcol//2, icol-mcol//2):
            img_new[i, j] = 0
            for k in range(mrow):
                for l in range(mcol):
                    if se[k][l]==1 and img_arr[i-mrow//2+k, j-mcol//2+l]==1:
                        img_new[i, j] = 1
                        break
                else:
                    continue
                break
    return img_new

def boundary_extraction(img_path, save_path, se):
    img_arr = img_to_arr(img_path)
    img_new = img_arr & ~erode(img_arr, se)
    arr_to_img(img_new, save_path)

def hole_filling(img_path, x0_path, save_path, se):
    img_arr = img_to_arr(img_path)
    x0 = img_to_arr(x0_path)
    neg_img_arr = ~img_arr
    while True:
        x1 = dilate(x0, se) & neg_img_arr
        if np.array_equal(x0,x1):
            break
        x0 = x1
    img_new = x0 | img_arr
    arr_to_img(img_new, save_path)

def connected(img_path, x0_path, save_path, se):
    img_arr = img_to_arr(img_path)
    x0 = img_to_arr(x0_path)
    while True:
        x1 = dilate(x0, se) & img_arr
        if np.array_equal(x0,x1):
            break
        x0 = x1
    arr_to_img(x1, save_path)

se = np.ones((3, 3), dtype=np.uint8)
boundary_extraction('Fig0914(a)(licoln from penny).tif','boundary_extraction.tif',se)
se = np.array([[0,1,0],
               [1,1,1],
               [0,1,0]])
hole_filling('Fig0916(a)(region-filling-reflections).tif','x0(512x512).tif','hole_filling.tif', se)
se = np.ones((3, 3), dtype=np.uint8)
connected('bone.tif','x0bone.tif','connected.tif', se)
